# Problema de Designação Desbalanceado

O problema de designação tem como objetivo designar tarefas a recursos. Por exemplo, tenhamos um conjunto de tarefas e um conjuto de equipes. Queremos definir qual equipe irá fazer qual projeto de modo a minimizar o custo total para realizar os projetos.

Índices:
- $i=1, ..., m$ refere-se as equipes;
- $j=1, ..., n$ refere-se aos tarefas;

Parâmetros:
- $c_{ij}$ custo para que a equipe $i$ faça a tarefa $j$.

### Objetivo: 

$$
\min \left(\sum_{i=1}^{m}\sum_{j=1}^{n}c_{ij}x_{ij}\right)
$$

### Sujeito as restrições:

#### Caso 1: Há mais equipes que tarefas; neste caso algumas equipes ficarão sem tarefas.

$$
\sum_{j=1}^{n}x_{ij} \leq 1, \;\; \forall i=1, ..., m \\
\sum_{i=1}^{m}x_{ij}=1, \;\; \forall j=1, ..., n \\
x_{ij}\geq 0
$$

#### Caso 2: Há mais tarefas que equipes; neste caso algumas tarefas não serão realizadas.

$$
\sum_{j=1}^{n}x_{ij} = 1, \;\; \forall i=1, ..., m \\
\sum_{i=1}^{m}x_{ij} \leq 1, \;\; \forall j=1, ..., n \\
x_{ij}\geq 0
$$

In [1]:
import gurobipy as gp

In [2]:
# Dados do problema
qtd_equipes = 6
qtd_tarefas = 6

vet_custo = [[108, 312, 144, 270, 160, 300],
             [108, 208, 144, 360, 180, 270],
             [108, 234, 180, 240, 180, 300],
             [144, 286, 180, 330, 240, 300],
             [132, 286, 144, 330, 180, 270],
             [120, 312, 144, 270, 220, 300]]

# Índices para as equipes
equipes = [f'Equipes_{i}' for i in range(1,qtd_equipes+1)]

# Índices para os projetos
tarefas = [f'Tarefas_{j}' for j in range(1,qtd_tarefas+1)]

In [3]:
custos = {(equipes[i],tarefas[j]):vet_custo[i][j] for i in range(qtd_equipes) for j in range(qtd_tarefas)}

In [5]:
# Cria modelo
m = gp.Model()

# Metodo para não imprimir o relatório
m.setParam(gp.GRB.Param.OutputFlag, 0)

# Variáveis de desição
x = m.addVars(equipes, tarefas, vtype=gp.GRB.BINARY)

# Função objetivo
m.setObjective(
    gp.quicksum(custos[i,j]*x[i,j] for i in equipes for j in tarefas),
    sense=gp.GRB.MINIMIZE
)

# Retrições de equipes
if qtd_equipes > qtd_tarefas:
    c1 = m.addConstrs(
        gp.quicksum(x[i,j] for j in tarefas) <= 1 for i in equipes
    )
else:
    c1 = m.addConstrs(
        gp.quicksum(x[i,j] for j in tarefas) == 1 for i in equipes
    )

# Retrições de tarefas
if qtd_tarefas > qtd_equipes:
    c2 = m.addConstrs(
        gp.quicksum(x[i,j] for i in equipes) <= 1 for j in tarefas
    )
else:
    c2 = m.addConstrs(
        gp.quicksum(x[i,j] for i in equipes) <= 1 for j in tarefas
    )

# Executa o modelo
m.optimize()

print(f'Custo total R$ {m.objVal}')

Custo total R$ 1166.0


In [6]:
# Mostra qual equira fará qual tarefa
for equipe in equipes:
    for tarefa in tarefas:
        if round(x[equipe,tarefa].X) == 1:
            print(f'A {equipe} fará o {tarefa}.')

A Equipes_1 fará o Tarefas_5.
A Equipes_2 fará o Tarefas_2.
A Equipes_3 fará o Tarefas_4.
A Equipes_4 fará o Tarefas_1.
A Equipes_5 fará o Tarefas_6.
A Equipes_6 fará o Tarefas_3.
